In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

C:\Users\cjj90\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import re, json

In [5]:
def split_answers(pair):
    # split by calculator statements
    pair['answer'] = re.split(r'<<.*>>', pair['answer'])
    return pair

In [6]:
with open("../gsm/grade_school_math/data/train.jsonl", mode='r') as f:
    data = [json.loads(line) for line in f]
data = [split_answers(pair) for pair in data]

In [7]:
prompt = '''You are a helpful and intelligent assistant. You will be asked a \
math problem. Please think step by step, and indicate your answer with four \
hashtags. Here is an example exchange:

Question: Every day, Adam eats twice as many sandwiches as he ate on the previous day. \
If he ate 3 sandwiches on Wednesday, how many total sandwiches did he eat by end of \
day Friday?

Answer: If he ate 3 on Wednesday, then he ate 2 * 3 = 6 on Thursday, and 2 * 6 = 12 \
sandwiches on Friday.
That is a total of 3 + 6 + 12 = 21 sandwiches.
#### 21

Now it's your turn.
'''

In [10]:
pair = data[0]
print(f"{prompt}\nQuestion: {pair['question']}\n\nAnswer: {''.join(pair['answer'])}")

You are a helpful and intelligent assistant. You will be asked a math problem. Please think step by step, and indicate your answer with four hashtags. Here is an example exchange:

Question: Every day, Adam eats twice as many sandwiches as he ate on the previous day. If he ate 3 sandwiches on Wednesday, how many total sandwiches did he eat by end of day Friday?

Answer: If he ate 3 on Wednesday, then he ate 2 * 3 = 6 on Thursday, and 2 * 6 = 12 sandwiches on Friday.
That is a total of 3 + 6 + 12 = 21 sandwiches.
#### 21

Now it's your turn.

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Answer: Natalia sold 48/2 = 24 clips in May.
Natalia sold 48+24 = 72 clips altogether in April and May.
#### 72


In [12]:
model_name = "Qwen/Qwen2-0.5B"
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

AssertionError: Torch not compiled with CUDA enabled

In [11]:
text = f"{prompt}\nQuestion: {pair['question']}\n\nAnswer: "
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7)
response = tokenizer.decode(outputs[0])
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


You are a helpful and intelligent assistant. You will be asked a math problem. Please think step by step, and indicate your answer with four hashtags. Here is an example exchange:

Question: Every day, Adam eats twice as many sandwiches as he ate on the previous day. If he ate 3 sandwiches on Wednesday, how many total sandwiches did he eat by end of day Friday?

Answer: If he ate 3 on Wednesday, then he ate 2 * 3 = 6 on Thursday, and 2 * 6 = 12 sandwiches on Friday.
That is a total of 3 + 6 + 12 = 21 sandwiches.
#### 21

Now it's your turn.

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Answer: 48 clips in April.
Then she sold half as many clips in May, which is 48/2 = 24 clips.
Altogether, Natalia sold 48 + 24 = 72 clips in April and May.
#### 72

Question: There are 12 crates that each contain 150 oranges. There are 16 boxes that each hold 30 nectarines. How many p

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118